# Alberto Rossettini
## Explanation of experiments on .txt

In [27]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [28]:
import os
import tensorflow as tf
import numpy as np

SEED = 777
tf.random.set_seed(SEED)  

cwd = os.getcwd()

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


### Generator
#### best scoring models without augmentation . By default augmentation set to False

In [29]:
# ImageDataGenerator
# ------------------

from tensorflow.keras.preprocessing.image import ImageDataGenerator

apply_data_augmentation = False

# Create training ImageDataGenerator object
# We need two different generators for images and corresponding masks
if apply_data_augmentation:
    train_img_data_gen = ImageDataGenerator(rotation_range=5,
                                            width_shift_range=5,
                                            height_shift_range=5,
                                            zoom_range=0.1,
                                            validation_split=0.2,
                                            horizontal_flip=True,
                                            vertical_flip=True,
                                            fill_mode='reflect',
                                            rescale=1./255)
    
    train_mask_data_gen = ImageDataGenerator(rotation_range=5,
                                             width_shift_range=5,
                                             height_shift_range=5,
                                             zoom_range=0.1,
                                             validation_split=0.2,
                                             horizontal_flip=True,
                                             vertical_flip=True,
                                             fill_mode='reflect',
                                             rescale=1./255,
                                             )
else:
    train_img_data_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
    train_mask_data_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Create validation and test ImageDataGenerator objects
valid_img_data_gen = ImageDataGenerator(rescale=1./255)
valid_mask_data_gen = ImageDataGenerator(rescale=1./255)
test_img_data_gen = ImageDataGenerator(rescale=1./255)
test_mask_data_gen = ImageDataGenerator(rescale=1./255)

## Dataset:
### make sure that the dataset is in the current folder, otherwise change dataset_dir to your own.

In [30]:
# Create generators to read images from dataset directory
# -------------------------------------------------------
dataset_dir = os.path.join('../input', 'annanddlimagessegmentation/Segmentation_Dataset')

# Batch size
bs = 16

# img shape
img_h = 256
img_w = 256

num_classes=2

# Training

training_dir = os.path.join(dataset_dir, 'training')
train_img_gen = train_img_data_gen.flow_from_directory(os.path.join(training_dir, 'images'),
                                                       batch_size=bs, 
                                                       class_mode=None, 
                                                       shuffle=True,
                                                       subset="training",
                                                       seed=SEED,
                                                       color_mode='rgb')  
train_mask_gen = train_mask_data_gen.flow_from_directory(os.path.join(training_dir, 'masks'),
                                                         batch_size=bs,
                                                         class_mode=None, 
                                                         shuffle=True,
                                                         subset="training",
                                                         seed=SEED,
                                                         color_mode='grayscale')
train_gen = zip(train_img_gen, train_mask_gen)

# Validation
validation_dir = os.path.join(dataset_dir, 'training')
valid_img_gen = train_img_data_gen.flow_from_directory(os.path.join(validation_dir, 'images'),
                                                       batch_size=bs, 
                                                       class_mode=None, 
                                                       shuffle=False,
                                                       subset="validation",
                                                       seed=SEED,color_mode='rgb')

valid_mask_gen = train_mask_data_gen.flow_from_directory(os.path.join(validation_dir, 'masks'),
                                                         batch_size=bs, 
                                                         class_mode=None, 
                                                         shuffle=False,
                                                         subset="validation",
                                                         seed=SEED, color_mode='grayscale')
valid_gen = zip(valid_img_gen, valid_mask_gen)

# Test
test_dir = os.path.join(dataset_dir, 'test')

Found 6118 images belonging to 1 classes.
Found 6118 images belonging to 1 classes.
Found 1529 images belonging to 1 classes.
Found 1529 images belonging to 1 classes.
Found 1234 images belonging to 1 classes.


"test_mask_gen = test_mask_data_gen.flow_from_directory(os.path.join(test_dir, 'masks'),\n                                                       target_size=(img_h, img_w),\n                                                       batch_size=bs, \n                                                       class_mode=None, # Because we have no class subfolders in this case\n                                                       shuffle=False,\n                                                       interpolation='bilinear',\n                                                       seed=SEED)"

In [31]:
# Create Dataset objects
# ----------------------

# Training
# --------
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, img_h, img_w, 1]))

def prepare_target(x_, y_):
    y_ = tf.cast(y_, tf.int32)
    return x_, y_

train_dataset = train_dataset.map(prepare_target)


# Repeat
train_dataset = train_dataset.repeat()

# Validation
# ----------
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, img_h, img_w, 1]))
valid_dataset = valid_dataset.map(prepare_target)

# Repeat
valid_dataset = valid_dataset.repeat()


## Rsidual blocks to be used in the net

In [32]:
def residual_block(blockInput, num_filters=16, batch_activate = False):
    x = BatchActivate(blockInput)
    x = convolution_block(x, num_filters, (3,3) )
    x = convolution_block(x, num_filters, (3,3), activation=False)
    x = tf.keras.layers.Add()([x, blockInput])
    if batch_activate:
        x = BatchActivate(x)
    return x

def BatchActivate(x):
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    return x

def convolution_block(x, filters, size, strides=(1,1), padding='same', activation=True):
    x = tf.keras.layers.Conv2D(filters, size, strides=strides, padding=padding)(x)
    if activation == True:
        x = BatchActivate(x)
    return x

# The model: 


In [33]:
# Create Model
# ------------
def create_model_noaug(start_f = 16):
    start_f = start_f
    input_shape = [img_h, img_w, 3]
    inputs = tf.keras.Input((img_h, img_w, 3))
    
    # Encoder
    # -------
    conv1 = tf.keras.layers.Conv2D(filters=start_f, #256_256_16
                                         kernel_size=(3, 3),
                                         strides=(1, 1),
                                         padding='same',
                                         input_shape=input_shape, activation='relu', 
                                         kernel_initializer="he_normal")(inputs)
    conv1 = residual_block(conv1,start_f)
    conv1 = residual_block(conv1,start_f, True)                                    
    pool1 = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(conv1) #128_128_16
    start_f = start_f*2
    drop1 = tf.keras.layers.Dropout(0.5*0.05)(pool1)
    conv2 = tf.keras.layers.Conv2D(filters=start_f,  #128_128_32
                                         kernel_size=(3, 3),
                                         strides=(1, 1),
                                         padding='same', activation='relu', 
                                   kernel_initializer="he_normal")(drop1)
    conv2 = residual_block(conv2,start_f)
    conv2 = residual_block(conv2,start_f, True)                             
    pool2 = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(conv2) #64_64_32
    drop2 = tf.keras.layers.Dropout(0.5)(pool2)
    start_f = start_f*2
    conv3 = tf.keras.layers.Conv2D(filters=start_f, #64_64_64
                                         kernel_size=(3, 3),
                                         strides=(1, 1),
                                         padding='same', activation='relu', 
                                   kernel_initializer="he_normal")(drop2)
    conv3 = residual_block(conv3,start_f)
    conv3 = residual_block(conv3,start_f, True)                                                         
    pool3 = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(conv3) #32_32_64
    drop3 = tf.keras.layers.Dropout(0.5)(pool3)
    start_f = start_f*2
    conv4 = tf.keras.layers.Conv2D(filters=start_f,  # 32_32_128
                                         kernel_size=(3, 3),
                                         strides=(1, 1),
                                         padding='same', activation='relu', 
                                   kernel_initializer="he_normal")(drop3)
    conv4 = residual_block(conv4,start_f)
    conv4 = residual_block(conv4,start_f, True) 
    pool4 = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(conv4) # 16_16_128
    drop4 = tf.keras.layers.Dropout(0.5)(pool4)

    start_f = start_f*2
    conv5 = tf.keras.layers.Conv2D(filters=start_f,  # 16_16_256
                                         kernel_size=(3, 3),
                                         strides=(1, 1),
                                         padding='same', activation='relu', 
                                         kernel_initializer="he_normal")(drop4)
    
    conv5 = residual_block(conv5,start_f)
    conv5 = residual_block(conv5,start_f, True)
    # Decoder + skip connection
    # -------
    
    up1 = tf.keras.layers.Conv2DTranspose(filters = start_f, #32_32_256
                                          kernel_size=(3, 3),
                                          strides=(2, 2),
                                          padding='same')(conv5)
    concat1 = tf.keras.layers.Concatenate()
    merge1 = concat1([up1,conv4])
    drop5 = tf.keras.layers.Dropout(0.5)(merge1)
    start_f = start_f // 2
    convt1 = tf.keras.layers.Conv2D(filters=start_f, #32_32_128
                                         kernel_size=(3, 3),
                                         strides=(1, 1),
                                         padding='same',activation='relu', 
                                    kernel_initializer="he_normal")(drop5)
    convt1 = residual_block(convt1,start_f)
    convt1 = residual_block(convt1,start_f, True)                                 
    up2 = tf.keras.layers.Conv2DTranspose(filters = start_f, #64_64_128 
                                          kernel_size=(3, 3),
                                          strides=(2, 2),
                                          padding='same', kernel_initializer="he_normal")(convt1)
    concat2 = tf.keras.layers.Concatenate()
    merge2 = concat2([up2,conv3])
    drop6 = tf.keras.layers.Dropout(0.5)(merge2)
    start_f = start_f // 2
    convt2 = tf.keras.layers.Conv2D(filters=start_f, #64_64_64
                                         kernel_size=(3, 3),
                                         strides=(1, 1),
                                         padding='same',activation='relu',
                                    kernel_initializer="he_normal")(drop6)
    convt2 = residual_block(convt2,start_f)
    convt2 = residual_block(convt2,start_f, True)                            
    
    up3 = tf.keras.layers.Conv2DTranspose(filters = start_f, #128_128_64
                                          kernel_size=(3, 3),
                                          strides=(2, 2),
                                          padding='same', kernel_initializer="he_normal")(convt2)
    concat3 = tf.keras.layers.Concatenate()
    merge3 = concat3([up3, conv2])
    drop7 = tf.keras.layers.Dropout(0.5)(merge3)
    start_f = start_f // 2                                      
    convt3 = tf.keras.layers.Conv2D(filters=start_f, #128_128_32
                                         kernel_size=(3, 3),
                                         strides=(1, 1),
                                         padding='same',activation='relu',
                                    kernel_initializer="he_normal")(drop7)
    convt3 = residual_block(convt3,start_f)
    convt3 = residual_block(convt3,start_f, True)    
    
    up4 = tf.keras.layers.Conv2DTranspose(filters = start_f, #256_256_32
                                          kernel_size=(3, 3),
                                          strides=(2, 2),
                                          padding='same', kernel_initializer="he_normal")(convt3)
    concat4 = tf.keras.layers.Concatenate()
    merge4 = concat4([up4,conv1])
    drop8 = tf.keras.layers.Dropout(0.5*0.05)(merge4)
    start_f = start_f // 2                                     
    convt4 = tf.keras.layers.Conv2D(filters=start_f, #256_256_16
                                         kernel_size=(3, 3),
                                         strides=(1, 1),
                                         padding='same',activation='relu',
                                    kernel_initializer="he_normal")(drop8)
    convt4 = residual_block(convt4,start_f)
    convt4 = residual_block(convt4,start_f, True) 
    
    # Prediction Layer
    # ----------------
    last = tf.keras.layers.Conv2D(filters=1, #256_256_2
                                     kernel_size=(1, 1),
                                     strides=(1, 1),
                                     padding='same',
                                     activation='sigmoid')(convt4)
    model = tf.keras.Model(inputs=inputs, outputs=last)
    
    return model

## run following cell only if you want to see the model

In [ ]:
img_w=256
img_h=256
model = create_model_noaug(32)
tf.keras.utils.plot_model(model, show_shapes=True, dpi=60)
model.summary()

## Dice loss function

In [34]:
def soft_dice_loss(y_true, y_pred, epsilon=1e-6): # fixed epsilon
    y_pred = tf.cast(y_pred , tf.float32)
    y_true = tf.cast(y_true , tf.float32)
    numerator = 2 * tf.reduce_sum(y_true * y_pred)
    denominator = tf.reduce_sum(y_true + y_pred)
    
    return 1 - tf.reduce_sum(numerator / (denominator + epsilon)) # average over classes and batch

# optimizer learning rate and evaluation metric

In [ ]:
lr = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------
def my_IoU(y_true, y_pred):
    # from pobability to predicted class {0, 1}
    y_pred = tf.cast(y_pred > 0.5, tf.float32) # when using sigmoid. Use argmax for softmax

    # A and B
    intersection = tf.reduce_sum(y_true * y_pred)
    # A or B
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
    # IoU
    return intersection / union

# load_weights:
## link to checkpoints of best performing iterations:
* ### https://polimi365-my.sharepoint.com/:f:/g/personal/10652220_polimi_it/Ery2R-9B0p5IpBn7eMuH40IBgao5Wxh6_NxTTwuU7NSl-w?e=QT79K5
### make sure you downloaded the checkpoints and put the folders in cwd

In [68]:
# ---------------------------------------
restore_model = True
if restore_model:
    model_noaug7300 = create_model_noaug(32)
    model_noaug7292 = create_model_noaug(32)
    model_noaug7330 = create_model_noaug(32)
    model_noaug7303 = create_model_noaug(32)
    model_noaug7370= create_model_noaug(32)
    model_noaug7317= create_model_noaug(32)

    model_noaug7300.compile(loss=soft_dice_loss, 
                  metrics=[my_IoU])  
    model_noaug7292.compile(loss=soft_dice_loss, 
                  metrics=[my_IoU])
    model_noaug7330.compile(loss=soft_dice_loss, 
                  metrics=[my_IoU])
    model_noaug7303.compile(loss=soft_dice_loss, 
                  metrics=[my_IoU])
    model_noaug7300.load_weights(os.path.join(
        cwd,'_7300', 'cp_03.ckpt'))  
    model_noaug7292.load_weights(os.path.join(
        cwd, '_7292', 'cp_03.ckpt')) 
    model_noaug7330.load_weights(os.path.join(
        cwd, '_7330', 'cp_03.ckpt')) 
    model_noaug7303.load_weights(os.path.join(
        cwd, '_7303', 'cp_02.ckpt')) 
    model_noaug7370.load_weights(os.path.join(
        cwd, '_7370', 'cp_03.ckpt'))
    model_noaug7317.load_weights(os.path.join(
        cwd, '_7317', 'cp_04.ckpt'))

In [37]:
def load(filename):
   np_image = Image.open(filename)
   np_image = np.array(np_image).astype('float32')/255
   np_image = np.expand_dims(np_image, axis=0)
   return np_image

In [38]:
def rle_encode(img):
      # Flatten column-wise
      pixels = img.T.flatten()
      pixels = np.concatenate([[0], pixels, [0]])
      runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
      runs[1::2] -= runs[::2]
      return ' '.join(str(x) for x in runs)

## sum of all the IoU scores of the models

In [48]:
tot = 2.9225

# Predictions

In [79]:
from tqdm import tqdm_notebook as tqdm
test_images = {}
for i in tqdm(os.listdir(os.path.join(test_dir,'images','img'))):
    image = load(os.path.join(test_dir,'images','img',i))
 
    prediction_noaug7300 = model_noaug7300.predict(image)
    prediction_noaug7292 = model_noaug7292.predict(image)
    prediction_noaug7330 = model_noaug7330.predict(image)
    prediction_noaug7303 = model_noaug7303.predict(image)
    prediction_noaug7370 = model_noaug7370.predict(image)
    prediction_noaug7317 = model_noaug7317.predict(image)

    prediction_noaug7300 = prediction_noaug7300[0]
    prediction_noaug7292 = prediction_noaug7292[0]
    prediction_noaug7330 = prediction_noaug7330[0]
    prediction_noaug7303 = prediction_noaug7303[0]
    prediction_noaug7370 = prediction_noaug7370[0]
    prediction_noaug7317 = prediction_noaug7317[0]

    prediction_img = np.zeros([image.shape[1],image.shape[2],3])
    prediction_img[np.where(tf.math.add_n([prediction_noaug7292*(0.7292/tot),prediction_noaug7370*(0.7370/tot),prediction_noaug7317*(0.7317/tot),prediction_noaug7300*(0.7300/tot),prediction_noaug7303*(0.7303/tot), prediction_noaug7330*(0.7330/tot)]) < 0.5)] = colors_dict[0]
    prediction_img[np.where(tf.math.add_n([prediction_noaug7292*(0.7292/tot),prediction_noaug7370*(0.7370/tot),prediction_noaug7317*(0.7317/tot),prediction_noaug7300*(0.7300/tot),prediction_noaug7303*(0.7303/tot), prediction_noaug7330*(0.7330/tot)]) > 0.5)] = colors_dict[1]

    rl = rle_encode(prediction_img)
    test_images[i[:-4]] = rl

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [44]:
import os
from datetime import datetime

def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(csv_fname, 'w') as f:

      f.write('ImageId,EncodedPixels,Width,Height\n')

      for key, value in results.items():
          f.write(key + ',' + str(value) + ',' + '256' + ',' + '256' + '\n')


# Submission

In [80]:
create_csv(test_images)